<a href="https://colab.research.google.com/github/RodericGuigoCorominas/datascience/blob/main/build_your_own_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
###DO NOT EDIT THIS CODE
################################################################################################################################

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# GPUs are 3x faster than CPU. Better to use if it is available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Loss Function
loss_function = nn.CrossEntropyLoss()

# This function returns the number of parameters in the model
def num_params(model):
  return sum([p.numel() for p in model.parameters()])

# Define a Training Function. This function will: compute the forward pass, backpropagate,
# update the weights, and repeat the steps for a given number of epochs. At each epoch, 
# it will output the training loss and test loss at every step
def train(epochs, model, trainloader, testloader, optimizer, loss_function):
  for epoch in range(epochs):
    loss_epoch = np.array([])
    train_correct, train_total = (0,0)
    test_correct, test_total = (0,0)
    for data, labels in trainloader:
      input_data = Variable(data).to(device)
      labels = labels.to(device)
      predict = model(input_data)
      loss = loss_function(predict, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_epoch = np.append(loss_epoch, loss.item())

    for data, labels in trainloader:
      input_data = Variable(data).to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          train_correct+=1
        train_total+=1
    
    train_accuracy = train_correct/train_total

    for data, labels in testloader:
      input_data = Variable(data).to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          test_correct+=1
        test_total+=1

    test_accuracy = test_correct/test_total    
  
    print('epoch [{}/{}], training loss:{:.4f}, training accuracy:{:.4f}, test loss:{:.4f}, test accuracy:{:.4f}'.format(epoch+1, epochs, np.mean(loss_epoch), train_accuracy, 0, test_accuracy))
################################################################################################################################

# **Load Dataset**

Available datasets are: MNIST, CIFAR10

## MNIST

In [6]:
# download and load data
batch_size = 512

# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=True, transform=transforms.Compose([
                                                transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                ])), batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), batch_size=batch_size, shuffle=True)

In [45]:
for data, labels in train_loader:
  print(data[0].size())
  break

torch.Size([1, 28, 28])


## SVHN

In [5]:
# download and load data
batch_size = 512

# download and transform train dataset
train_dataset = datasets.SVHN(root='/content/gdrive/.shortcut-targets-by-id/1IrVCGk9CH8ZYDWEH4w96PenFpHYJx6Cs/Easy Nets', 
                                                         split='train', transform=None, download=True)
train_dataset = [(transforms.ToTensor()(img[0]), img[1]) for img in train_dataset]
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_dataset = datasets.SVHN(root='/content/gdrive/.shortcut-targets-by-id/1IrVCGk9CH8ZYDWEH4w96PenFpHYJx6Cs/Easy Nets',
                                                        split='test', transform=None, download=True)
test_dataset = [(transforms.ToTensor()(img[0]), img[1]) for img in test_dataset]
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/182040794 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
### Test Dimensions
iterable = iter(train_loader)
print(iterable.next()[0].shape)
print(iterable.next()[1].shape)

# **Build a Network and Define Hyperparameters**

In [32]:
learning_rate = 10e-3
weight_decay = 10e-5
n_epochs = 10

# neural network
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    ### Define Layers
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)

  def forward(self, x):
    ###Feed Forward
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output
    

In [34]:
# Every time you edit the neural network, you'll have to update this cell
# Create model object
model = NeuralNetwork().to(device)

# Loads Adam optimizer, which implements a version of gradient descent
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [35]:
# check the structure of your network
print(model)

NeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)


In [49]:
# apply your model to a single input
model(torch.rand(1,1,28,28, device=device))

tensor([[-2.3417, -2.4135, -2.9292, -2.6968, -1.8989, -2.5332, -2.7527, -2.1700,
         -1.9680, -1.9197]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [46]:
torch.rand(1,28,28, device=device).size()

torch.Size([1, 28, 28])

# **Train and Validate**

In [39]:
train(n_epochs, model, train_loader, test_loader, optimizer, loss_function)

epoch [1/10], training loss:0.5127, training accuracy:0.9333, test loss:0.0000, test accuracy:0.9361


KeyboardInterrupt: ignored

# **No Batch Normalization**

In [ ]:
# neural network
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2)
    self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.avg = nn.AdaptiveAvgPool2d(4)
    self.fc = nn.Linear(1024,10)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv3(x))
    x = torch.relu(self.conv4(x))
    x = torch.relu(self.conv5(x))
    x = self.maxpool(x)
    x = self.avg(x)
    x = x.view(-1,1024)
    x = self.fc(x)
    x = self.softmax(x)
    return(x)

model = CNN().to(device)

# **Best Model**

In [ ]:
# neural network
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
    self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(64)
    self.avg = nn.AdaptiveAvgPool2d(4)
    self.fc = nn.Linear(1024,10)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = self.maxpool(x)
    x = self.bn1(x)
    x = torch.relu(self.conv3(x))
    x = torch.relu(self.conv4(x))
    x = torch.relu(self.conv5(x))
    x = self.maxpool(x)
    x = self.bn2(x)
    x = self.avg(x)
    x = x.view(-1,1024)
    x = self.fc(x)
    x = self.softmax(x)
    return(x)

model = CNN().to(device)